In [27]:
CLIENT_ID = "hZ-F6QLaGJ6VBg" 
CLIENT_SECRET = "MFC8FXTlXjm4jtG60bm9Qs-PPls"

In [28]:
USER_AGENT = "python:<test0.1> (by /u/Dealsy23)"

In [29]:
USERNAME = "Dealsy23" 
PASSWORD = "Outbreak23"

In [30]:
import requests
def login(username, password):
    if password is None:
        password = getpass.getpass("Enter reddit password for user {}: ".format(username))    
    headers = {"User-Agent": USER_AGENT}
    # Setup an auth object with our credentials
    client_auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
    # Make a post request to the access_token endpoint
    post_data = {"grant_type": "password", "username": username, "password": password}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth,     
                             data=post_data, headers=headers) 
    return response.json()

In [31]:
token = login(USERNAME, PASSWORD)


In [32]:
token

{'access_token': 'K9NuIv1-0S3zDIVj6oyOD1dw83Y',
 'expires_in': 3600,
 'scope': '*',
 'token_type': 'bearer'}

In [33]:
subreddit = "worldnews"

In [34]:
url = "https://oauth.reddit.com/r/{}".format(subreddit)

In [35]:
headers = {"Authorization": "bearer {}".format(token['access_token']), 
"User-Agent": USER_AGENT}

In [46]:
response = requests.get(url, headers=headers)

In [47]:
result = response.json()

In [48]:
for story in result['data']['children']: 
    print(story['data']['title'])

Russia considers banning sale of cigarettes to anyone born after 2015
Swiss Muslim girls must swim with boys
Report: Russia spread fake news and disinformation in Sweden - Russia has coordinated a campaign over the past 2years to influence Sweden’s decision making by using disinformation, propaganda and false documents, according to a report by researchers at The Swedish Institute of International Affairs.
100% of Dutch Trains Now Run on Wind Energy. The Netherlands met its renewable energy goals a year ahead of time.
Legal challenge against UK’s sweeping surveillance laws quickly crowdfunded
A 1,000-foot-thick ice block about the size of Delaware is snapping off of Antarctica
The U.S. dropped an average of 72 bombs every day — the equivalent of three an hour — in 2016, according to an analysis of American strikes around the world. U.S. Bombed Iraq, Syria, Pakistan, Afghanistan, Libya, Yemen, Somalia in 2016
The German government is investigating a recent surge in fake news following c

In [49]:
from time import sleep

In [55]:
def get_links(subreddit, token, n_pages=5):
    stories = []
    after = None
    for page_number in range(n_pages):
        # Sleep before making calls to avoid going over the API limit
        sleep(2)
        # Setup headers and make call, just like in the login function
        headers = {"Authorization": "bearer {}".format(token['access_token']), "User-Agent": USER_AGENT} 
        url = "https://oauth.reddit.com/r/{}?limit=100". format(subreddit)
        if after:
            # Append cursor for next page, if we have one
            url += "&after={}".format(after)
        response = requests.get(url, headers=headers)
        result = response.json()
        # Get the new cursor for the next loop
        after = result['data']['after']
        # Add all of the news items to our stories list
        for story in result['data']['children']:
            stories.append((story['data']['title'], story['data']['url'], story['data']['score']))
    return stories

In [56]:
stories = get_links("worldnews", token)

In [5]:
import os 
data_folder = os.path.join("data", "websites", "raw")

In [59]:
import hashlib

In [60]:
number_errors = 0


In [66]:
for title, url, score in stories:
    output_filename = hashlib.md5(url.encode()).hexdigest() 
    fullpath = os.path.join(data_folder, output_filename + ".txt")
    try: 
        response = requests.get(url) 
        data = response.text 
        with open(fullpath, 'w') as outf: 
            outf.write(data)
    except Exception as e:
        number_errors += 1
        # You can use this to view the errors, if you are getting too many:
        #raise

In [67]:
number_errors

503

In [7]:
filenames = [os.path.join(data_folder, filename) for filename in os.listdir(data_folder)]

In [16]:
text_output_folder = os.path.join("data", "websites", "textonly")

In [17]:
import lxml
from lxml import etree

In [18]:
skip_node_types = ["script", "head", "style", etree.Comment]


In [19]:
parser = etree.HTMLParser()

def get_text_from_file(filename):
    with open(filename) as inf:
        html_tree = etree.parse(inf, parser) 
    return get_text_from_node(html_tree.getroot())

In [20]:
def get_text_from_node(node):
    if len(node) == 0: 
        # No children, just return text from this item
        if node.text: 
            return node.text 
        else:
            return ""
    else:
        # This node has children, return the text from it:
        results = (get_text_from_node(child)
                   for child in node
                   if child.tag not in skip_node_types)
    result = str.join("\n", (r for r in results if len(r) > 1))
    if len(result) >= 100:
        return result
    else:
        return ""

In [21]:
for filename in os.listdir(data_folder):
    text = get_text_from_file(os.path.join(data_folder, filename)) 
    with open(os.path.join(text_output_folder, filename), 'w') as outf: 
        outf.write(text)

In [22]:
from sklearn.cluster import KMeans

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
from sklearn.pipeline import Pipeline
n_clusters = 10 
pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4)),
                                     ('clusterer', KMeans(n_clusters=n_clusters)) ])

In [25]:

documents = [open(os.path.join(text_output_folder, filename)).read() for filename in os.listdir(text_output_folder)]

In [26]:
pipeline.fit(documents)
labels = pipeline.predict(documents)

In [27]:
from collections import Counter
c = Counter(labels) 
for cluster_number in range(n_clusters): 
    print("Cluster {} contains {} samples".format(cluster_number, c[cluster_number]))

Cluster 0 contains 42 samples
Cluster 1 contains 37 samples
Cluster 2 contains 31 samples
Cluster 3 contains 26 samples
Cluster 4 contains 28 samples
Cluster 5 contains 13 samples
Cluster 6 contains 43 samples
Cluster 7 contains 25 samples
Cluster 8 contains 209 samples
Cluster 9 contains 37 samples


In [28]:
pipeline.named_steps['clusterer'].inertia_

414.5540395641533

In [29]:
inertia_scores = [] 
n_cluster_values = list(range(2, 20)) 
for n_clusters in n_cluster_values: 
    cur_inertia_scores = [] 
    X = TfidfVectorizer(max_df=0.4).fit_transform(documents) 
    for i in range(10): 
        km = KMeans(n_clusters=n_clusters).fit(X) 
        cur_inertia_scores.append(km.inertia_) 
        inertia_scores.append(cur_inertia_scores)

In [30]:
n_clusters

19

In [31]:
n_clusters = 6 
pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4)),
                     ('clusterer', KMeans(n_clusters=n_clusters)) ])
pipeline.fit(documents) 
labels = pipeline.predict(documents)

In [32]:
n_clusters

6

In [33]:
terms = pipeline.named_steps['feature_extraction'].get_feature_names()

In [34]:
c = Counter(labels)

In [35]:
for cluster_number in range(n_clusters): 
    print("Cluster {} contains {} samples".format(cluster_number, c[cluster_number]))
    print(" Most important terms")
    centroid = pipeline.named_steps['clusterer'].cluster_centers_[cluster_number]
    most_important = centroid.argsort()
    for i in range(5):
        term_index = most_important[-(i+1)]
        print(" {0}) {1} (score: {2:.4f})".format(i+1, terms[term_index], centroid[term_index]))

Cluster 0 contains 22 samples
 Most important terms
 1) iran (score: 0.1932)
 2) iranian (score: 0.1310)
 3) shots (score: 0.1198)
 4) mahan (score: 0.1109)
 5) navy (score: 0.1108)
Cluster 1 contains 36 samples
 Most important terms
 1) click (score: 0.1833)
 2) adblock (score: 0.1786)
 3) icon (score: 0.1166)
 4) site (score: 0.1012)
 5) getty (score: 0.0796)
Cluster 2 contains 308 samples
 Most important terms
 1) she (score: 0.0185)
 2) her (score: 0.0178)
 3) china (score: 0.0174)
 4) us (score: 0.0171)
 5) government (score: 0.0141)
Cluster 3 contains 43 samples
 Most important terms
 1) kabul (score: 0.0654)
 2) afghanistan (score: 0.0566)
 3) taliban (score: 0.0558)
 4) killed (score: 0.0509)
 5) pakistan (score: 0.0490)
Cluster 4 contains 45 samples
 Most important terms
 1) washington (score: 0.1076)
 2) industry (score: 0.0755)
 3) tuesday (score: 0.0742)
 4) your (score: 0.0719)
 5) tax (score: 0.0699)
Cluster 5 contains 37 samples
 Most important terms
 1) israeli (score: 

In [36]:
X = pipeline.transform(documents)

In [37]:
from scipy.sparse import csr_matrix

In [38]:
import numpy as np
def create_coassociation_matrix(labels):
    rows = [] 
    cols = []
    unique_labels = set(labels) 
    for label in unique_labels:
        indices = np.where(labels == label)[0]
        for index1 in indices:
            for index2 in indices:
                rows.append(index1)
                cols.append(index2)
    data = np.ones((len(rows),)) 
    return csr_matrix((data, (rows, cols)), dtype='float')

In [39]:
C = create_coassociation_matrix(labels)

In [40]:
C

<491x491 sparse matrix of type '<class 'numpy.float64'>'
	with 101887 stored elements in Compressed Sparse Row format>

In [41]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [42]:
mst = minimum_spanning_tree(C)

In [43]:
mst = minimum_spanning_tree(-C)

In [44]:
pipeline.fit(documents) 
labels2 = pipeline.predict(documents) 
C2 = create_coassociation_matrix(labels2) 
C_sum = (C + C2) / 2

In [45]:
mst = minimum_spanning_tree(-C_sum) 
mst.data[mst.data > -1] = 0

In [46]:
from scipy.sparse.csgraph import connected_components 
number_of_clusters, labels = connected_components(mst)

In [47]:
from sklearn.base import BaseEstimator, ClusterMixin 
class EAC(BaseEstimator, ClusterMixin):
    def __init__(self, n_clusterings=10, cut_threshold=0.5, n_clusters_range=(3, 10)): 
        self.n_clusterings = n_clusterings
        self.cut_threshold = cut_threshold
        self.n_clusters_range = n_clusters_range

    def fit(self, X, y=None):
        C = sum((create_coassociation_matrix(self._single_clustering(X))
                 for i in range(self.n_clusterings)))
        mst = minimum_spanning_tree(-C)
        mst.data[mst.data > -self.cut_threshold] = 0
        mst.eliminate_zeros()
        self.n_components, self.labels_ = connected_components(mst)
        return self
 
    def _single_clustering(self, X):
        n_clusters = np.random.randint(*self.n_clusters_range)
        km = KMeans(n_clusters=n_clusters)
        return km.fit_predict(X)
 
    def fit_predict(self, X):
        self.fit(X)
        return self.labels_

In [48]:
pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4)),
                     ('clusterer', EAC()) ])

In [49]:
vec = TfidfVectorizer(max_df=0.4) 
X = vec.fit_transform(documents)

In [50]:
from sklearn.cluster import MiniBatchKMeans 
mbkm = MiniBatchKMeans(random_state=14, n_clusters=3)

In [51]:
batch_size = 10 
for iteration in range(int(X.shape[0] / batch_size)): 
    start = batch_size * iteration 
    end = batch_size * (iteration + 1) 
    mbkm.partial_fit(X[start:end])

In [52]:
labels = mbkm.predict(X)

In [53]:
labels

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2,

In [54]:
class PartialFitPipeline(Pipeline):
    def partial_fit(self, X, y=None):
        Xt = X
        for name, transform in self.steps[:-1]:
            Xt = transform.transform(Xt)
        return self.steps[-1][1].partial_fit(Xt, y=y)

In [55]:
from sklearn.feature_extraction.text import HashingVectorizer

pipeline = PartialFitPipeline([('feature_extraction', HashingVectorizer()),
                               ('clusterer', MiniBatchKMeans(random_state=14, n_clusters=3)) ])
batch_size = 10 
for iteration in range(int(len(documents) / batch_size)): 
    start = batch_size * iteration 
    end = batch_size * (iteration + 1)
    pipeline.partial_fit(documents[start:end]) 
labels = pipeline.predict(documents)

In [56]:
labels

array([0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 2, 0, 1, 0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2, 1, 1, 2, 0, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1,
       2, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       2, 1,